In [2]:
# from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
import torch
# from transformers import OPTForCausalLM, AutoModelForCausalLM, AutoTokenizer

from datasets import load_dataset
from datasets.iterable_dataset import IterableDataset
# from transformers import OPTForCausalLM
# from chemlactica.utils.model_utils import load_model
# from chemlactica.utils.utils import get_tokenizer
import scipy
import numpy as np
import numpy
import random
from sklearn.metrics import root_mean_squared_error
from rdkit import Chem

In [3]:
torch.manual_seed(42)
random.seed(42)
numpy.random.seed(42)

In [9]:
chemlactica_tokenizer = AutoTokenizer.from_pretrained("/auto/home/menuab/code/ChemLactica/chemlactica/tokenizer/ChemLacticaTokenizer66")
chemma_tokenizer = AutoTokenizer.from_pretrained("/auto/home/menuab/code/ChemLactica/chemlactica/tokenizer/GemmaTokenizer")
galactica_tokenizer = AutoTokenizer.from_pretrained("facebook/galactica-125m")
len(chemlactica_tokenizer), len(chemma_tokenizer), len(galactica_tokenizer)

NameError: name 'AutoTokenizer' is not defined

In [77]:
galactica_model = AutoModelForCausalLM.from_pretrained(
    "facebook/galactica-1.3b").to("cuda:0").eval()
galactica_model.device, galactica_model.dtype


/auto/home/menuab/miniforge3/envs/gemma_env_new/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


(device(type='cuda', index=0), torch.float32)

In [11]:
model_125m_18k_1f28 = "/nfs/dgx/raid/chem/checkpoints/facebook/galactica-125m/1f289ff103034364bd27e1c3/checkpoint-18000/"
chemlactcia_model = AutoModelForCausalLM.from_pretrained(
    model_125m_18k_1f28 ,torch_dtype=torch.bfloat16, attn_implementation="flash_attention_2"
).to("cuda:0").eval()
chemlactcia_model.device, chemlactcia_model.dtype

(device(type='cuda', index=0), torch.bfloat16)

In [26]:
prompt = f"The IUPAC name of cortisol is:"
prompt = chemlactica_tokenizer(prompt, return_tensors="pt").to(chemlactcia_model.device).input_ids
output = chemlactcia_model.generate(prompt, do_sample=False, max_new_tokens=50)
chemlactica_tokenizer.decode(output[0])

'The IUPAC name of cortisol is: 1000 mg/kg bw[/PROPERTY][PROPERTY]Non-Human Toxicity Excerpts /LABORATORY ANIMALS: Acute Exposure/... The effects of a dose of 1000 mg/kg bw bw per'

In [25]:
prompt = f"The IUPAC name of cortisol is:"
prompt = galactica_tokenizer(prompt, return_tensors="pt").to(galactica_model.device).input_ids
output = galactica_model.generate(prompt, do_sample=False, max_new_tokens=50)
galactica_tokenizer.decode(output[0])

'The IUPAC name of cortisol is:\n\n1-[(2-chlorophenyl)methyl]-3-(2-methoxyphenyl)urea\n\n### InChI\n\nInChI=1S/C16H15ClN2O2/c1-20-1'

In [32]:
prompt = f"[START_SMILES]CC(C)(C)C(=O)N(CC1=NC(=CS1)C(=O)OC)C2CCCCC2[END_SMILES]"
prompt = chemlactica_tokenizer(prompt, return_tensors="pt").to(chemlactcia_model.device).input_ids
output = chemlactcia_model.generate(prompt, do_sample=False, max_new_tokens=2000)
chemlactica_tokenizer.decode(output[0])

'[START_SMILES]CC(C)(C)C(=O)N(CC1=NC(=CS1)C(=O)OC)C2CCCCC2[END_SMILES][HEAVYATOMCOUNT]23[/HEAVYATOMCOUNT][NUMHETEROATOMS]6[/NUMHETEROATOMS][TPSA]58.97[/TPSA][NUMAROMATICHETEROCYCLES]0[/NUMAROMATICHETEROCYCLES][NUMHDONORS]0[/NUMHDONORS][NUMHACCEPTORS]5[/NUMHACCEPTORS][NUMALIPHATICRINGS]2[/NUMALIPHATICRINGS][SAS]3.34[/SAS][RINGCOUNT]2[/RINGCOUNT][FRACTIONCSP3]0.71[/FRACTIONCSP3][NUMROTATABLEBONDS]3[/NUMROTATABLEBONDS][NUMAROMATICRINGS]0[/NUMAROMATICRINGS][CLOGP]3.39[/CLOGP][WEIGHT]338.17[/WEIGHT][QED]0.59[/QED][FORMULA]C17H26N2O3S[/FORMULA][NOCOUNT]5[/NOCOUNT][NHOHCOUNT]0[/NHOHCOUNT][NUMSATURATEDRINGS]1[/NUMSATURATEDRINGS][NUMAROMATICCARBOCYCLES]0[/NUMAROMATICCARBOCYCLES][NUMSATURATEDHETEROCYCLES]0[/NUMSATURATEDHETEROCYCLES][NUMALIPHATICCARBOCYCLES]1[/NUMALIPHATICCARBOCYCLES][IUPAC]methyl (4Z)-2-[[cyclohexyl(2,2-dimethylpropanoyl)amino]methyl]-1,3-thiazole-4-carboxylate[/IUPAC][NUMALIPHATICHETEROCYCLES]1[/NUMALIPHATICHETEROCYCLES][NUMSATURATEDCARBOCYCLES]1[/NUMSATURATEDCARBOCYCLES]</s>'

In [33]:
prompt = f"[START_SMILES]CC(C)(C)C(=O)N(CC1=NC(=CS1)C(=O)OC)C2CCCCC2[END_SMILES][IUPAC]"
prompt = chemlactica_tokenizer(prompt, return_tensors="pt").to(chemlactcia_model.device).input_ids
output = chemlactcia_model.generate(prompt, do_sample=False, max_new_tokens=2000)
chemlactica_tokenizer.decode(output[0])

'[START_SMILES]CC(C)(C)C(=O)N(CC1=NC(=CS1)C(=O)OC)C2CCCCC2[END_SMILES][IUPAC]methyl (4Z)-2-[[cyclohexyl(2,2-dimethylpropanoyl)amino]methyl]-1,3-thiazole-4-carboxylate[/IUPAC][TPSA]58.97[/TPSA][FORMULA]C17H26N2O3S[/FORMULA][SAS]3.24[/SAS][NUMHACCEPTORS]5[/NUMHACCEPTORS][RINGCOUNT]2[/RINGCOUNT][FRACTIONCSP3]0.71[/FRACTIONCSP3][NUMAROMATICRINGS]0[/NUMAROMATICRINGS][WEIGHT]338.17[/WEIGHT][NUMSATURATEDRINGS]1[/NUMSATURATEDRINGS][QED]0.59[/QED][HEAVYATOMCOUNT]23[/HEAVYATOMCOUNT][NUMAROMATICHETEROCYCLES]0[/NUMAROMATICHETEROCYCLES][CLOGP]3.39[/CLOGP][NUMHDONORS]0[/NUMHDONORS][NUMHETEROATOMS]6[/NUMHETEROATOMS][NUMROTATABLEBONDS]3[/NUMROTATABLEBONDS][NUMALIPHATICCARBOCYCLES]1[/NUMALIPHATICCARBOCYCLES][NOCOUNT]5[/NOCOUNT][NUMALIPHATICRINGS]2[/NUMALIPHATICRINGS][NHOHCOUNT]0[/NHOHCOUNT][NUMAROMATICCARBOCYCLES]0[/NUMAROMATICCARBOCYCLES][NUMSATURATEDHETEROCYCLES]0[/NUMSATURATEDHETEROCYCLES][NUMSATURATEDCARBOCYCLES]1[/NUMSATURATEDCARBOCYCLES][NUMALIPHATICHETEROCYCLES]1[/NUMALIPHATICHETEROCYCLES]</s>'

In [31]:
prompt = f"The IUPAC name of [START_SMILES]CC(C)(C)C(=O)N(CC1=NC(=CS1)C(=O)OC)C2CCCCC2[END_SMILES] is:"
prompt = galactica_tokenizer(prompt, return_tensors="pt").to(galactica_model.device).input_ids
output = galactica_model.generate(prompt, do_sample=False, max_new_tokens=50)
galactica_tokenizer.decode(output[0])

'The IUPAC name of [START_SMILES]CC(C)(C)C(=O)N(CC1=NC(=CS1)C(=O)OC)C2CCCCC2[END_SMILES] is:\n\n### Isomeric SMILES\n\n[START_I_SMILES]CC(C)(C)C(=O)N(CC1=NC(=CS1)C(=O)OC)C2CCCCC2[END_I_SMILES]\n\n### Molecular Formula'

In [ ]:
methyl 2-[[cyclohexyl-(2,2-dimethylpropanoyl)]amino]methyl]thiazole-4-carboxylate

In [ ]:
methyl 2-[[cyclohexyl(2,2-dimethylpropanoyl)amino]methyl]-1,3-thiazole-4-carboxylate 

In [ ]:
methyl (4Z)-2-[[cyclohexyl(2,2-dimethylpropanoyl)amino]methyl]-1,3-thiazole-4-carboxylate

In [71]:
prompt = f"Here is a SMILES formula: [START_I_SMILES]CCOC(=O)[C@H](CCc1ccccc1)N[C@@H](C)C(=O)N2CCC[C@H]2C(O)=O[END_I_SMILES]\n\nQuestion: Will the chemical compound penetrate the blood-brain barrier?\n\nAnswer:"
prompt = galactica_tokenizer(prompt, return_tensors="pt").to(galactica_model.device).input_ids
output = galactica_model.generate(prompt, do_sample=False, max_new_tokens=50)
galactica_tokenizer.decode(output[0])

'Here is a SMILES formula: [START_I_SMILES]CCOC(=O)[C@H](CCc1ccccc1)N[C@@H](C)C(=O)N2CCC[C@H]2C(O)=O[END_I_SMILES]\n\nQuestion: 1-(2-furylmethyl)-3-(2-thienylmethyl)urea.\n\n### Computed Properties\n\n| Property Name | Property Value\n| --- | ----------- |\n| Molecular Weight | '

In [4]:
dataset = load_dataset('gayane/BBBp')

In [5]:
dataset['test']['activity']

[0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0

In [37]:
dataset['test']['smiles']

['CCOC(=O)[C@H](CCc1ccccc1)N[C@@H](C)C(=O)N2CCC[C@H]2C(O)=O',
 'FC(F)(F)[C@]1(OC(=O)Nc2ccc(Cl)cc12)C#CC3CC3',
 'CN1C[C@@H](C[C@H]2[C@H]1Cc3c[nH]c4cccc2c34)C(=O)N[C@]5(C)O[C@@]6(O)[C@@H]7CCCN7C(=O)[C@H](Cc8ccccc8)N6C5=O',
 'CN1CCC[C@H]1c2cccnc2',
 'CN1CCC(=CC1)c2ccccc2',
 'C[C@@H](CN1CC(=O)NC(=O)C1)N2CC(=O)NC(=O)C2',
 'CN(C)N\\N=C/1N=CN=C1C(N)=O',
 'CCC1(CC)C(=O)NCC(C)C1=O',
 'Nc1nc(NC2CC2)c3ncn([C@@H]4C[C@H](CO)C=C4)c3n1',
 'c1(c(cc2n(ncc2c1)CCN)Cl)Cl',
 'c1c2CCOc2c(cc1OC)CN[C@@H]1CCCN[C@H]1c1ccccc1',
 'CC(C)N1CCC(CC1)N(C(=O)Cc2ccccc2)c3ccc(Cl)cc3',
 'n1(c(c2nc[nH]c2n(c1=O)C)=O)C',
 'c1(nccc(c1)c1nc([nH]n1)N)N(C)C',
 'CN(C)Cc1oc(cc1)CSCCNC=1NC=C(CN1)Cc1cnc(cc1)C',
 'OC(=O)C1=CC(=O)c2ccccc2N1',
 'CNC1(CCCCC1=O)c2ccccc2Cl',
 'CN(C1CCCC[C@H]1N2CCCC2)C(=O)Cc3ccc(Cl)c(Cl)c3',
 'CN1C(=C(\\O)Nc2ccccn2)/C(=O)c3sccc3[S]1(=O)=O',
 '[NH]C(CC(C)C([N@@](C(C)(C)C)C(N)(C)N)(C)C)c1c(c(c[nH+][o+]1)C)[O-]',
 'Cc1nccc2c1[nH]c3ccccc23',
 'CC1=C2NC3=CC(=O)C=CC3=C2C=CN1',
 'COc1cc2CCN(C)C3CC4(C=CC(=O)C=C4)c

In [94]:
mol = Chem.MolFromSmiles("CCOC(=O)[C@H](CCc1ccccc1)N[C@@H](C)C(=O)N2CCC[C@H]2C(O)=O")
std_smiles1 = Chem.MolToSmiles(mol)
std_smiles2 = Chem.MolToSmiles(mol, doRandom=True)
kek_smiles = Chem.MolToSmiles(mol, kekuleSmiles=True)
can_smiles = Chem.MolToSmiles(mol, canonical=True)
iso_smiles = Chem.MolToSmiles(mol, isomericSmiles=True)
kek_iso_smiles = Chem.MolToSmiles(mol, isomericSmiles=True, kekuleSmiles=True)
(std_smiles1, std_smiles2, can_smiles, iso_smiles, kek_smiles, kek_iso_smiles)

('CCOC(=O)[C@H](CCc1ccccc1)N[C@@H](C)C(=O)N1CCC[C@H]1C(=O)O',
 'C([C@H](N[C@H](C(=O)N1CCC[C@H]1C(=O)O)C)C(OCC)=O)Cc1ccccc1',
 'CCOC(=O)[C@H](CCc1ccccc1)N[C@@H](C)C(=O)N1CCC[C@H]1C(=O)O',
 'CCOC(=O)[C@H](CCc1ccccc1)N[C@@H](C)C(=O)N1CCC[C@H]1C(=O)O',
 'CCOC(=O)[C@H](CCC1=CC=CC=C1)N[C@@H](C)C(=O)N1CCC[C@H]1C(=O)O',
 'CCOC(=O)[C@H](CCC1=CC=CC=C1)N[C@@H](C)C(=O)N1CCC[C@H]1C(=O)O')

In [84]:
tokenizer = galactica_tokenizer
ground_truths, gens, diffs = [],[],[]
invalids = 0
for sample in dataset['test']:
    ground_truth = round(sample['activity'], 2)
    mol = Chem.MolFromSmiles(sample['smiles'])
    iso_smiles = Chem.MolToSmiles(mol, isomericSmiles=True)
    prompt = f"Here is a SMILES formula\n\n[START_I_SMILES]{iso_smiles}[END_I_SMILES]\n\nQuestion: Will the chemical compound penetrate the blood-brain barrier?\n\nAnswer:"
    len_prompt = len(prompt)
    prompt = tokenizer(prompt, return_tensors="pt").to(galactica_model.device)
    out = galactica_model.generate(prompt.input_ids, do_sample=False, eos_token_id=tokenizer.encode('</s>')[0], max_new_tokens=40)
    out = tokenizer.batch_decode(out)[0]
    try:
        gen = out[out.find("Answer: ") + len("Answer: "):out.find("</s>")]
        gen = 0.0 if gen == 'No' else 1.0
        diff = abs(ground_truth - gen)
        print("GT:", ground_truth, "Gen:", gen, "diff:", round(diff,2), out )
        ground_truths.append(ground_truth)
        gens.append(gen)
        diffs.append(diff)
    except:
        print('***')
        print("GT:", ground_truth, out)
        invalids += 1
        pass

GT: 0.0 Gen: 1.0 diff: 1.0 Here is a SMILES formula

[START_I_SMILES]CCOC(=O)[C@H](CCc1ccccc1)N[C@@H](C)C(=O)N1CCC[C@H]1C(=O)O[END_I_SMILES]

Question: Will the chemical compound penetrate the blood-brain barrier?

Answer: Yes</s>
GT: 0.0 Gen: 1.0 diff: 1.0 Here is a SMILES formula

[START_I_SMILES]O=C1Nc2ccc(Cl)cc2[C@@](C#CC2CC2)(C(F)(F)F)O1[END_I_SMILES]

Question: Will the chemical compound penetrate the blood-brain barrier?

Answer: Yes</s>
GT: 0.0 Gen: 1.0 diff: 1.0 Here is a SMILES formula

[START_I_SMILES]CN1C[C@H](C(=O)N[C@]2(C)O[C@@]3(O)[C@@H]4CCCN4C(=O)[C@H](Cc4ccccc4)N3C2=O)C[C@@H]2c3cccc4[nH]cc(c34)C[C@H]21[END_I_SMILES]

Question: Will the chemical compound penetrate the blood-brain barrier?

Answer: Yes</s>
GT: 1.0 Gen: 1.0 diff: 0.0 Here is a SMILES formula

[START_I_SMILES]CN1CCC[C@H]1c1cccnc1[END_I_SMILES]

Question: Will the chemical compound penetrate the blood-brain barrier?

Answer: Yes</s>
GT: 1.0 Gen: 1.0 diff: 0.0 Here is a SMILES formula

[START_I_SMILES]CN1CC=

[19:16:51] WARNING: not removing hydrogen atom without neighbors


GT: 0.0 Gen: 1.0 diff: 1.0 Here is a SMILES formula

[START_I_SMILES]CCN(CC)CCOC(=O)C1(C2CCCCC2)CCCCC1[END_I_SMILES]

Question: Will the chemical compound penetrate the blood-brain barrier?

Answer: Yes</s>
GT: 0.0 Gen: 1.0 diff: 1.0 Here is a SMILES formula

[START_I_SMILES]C[C@H](CCC(=O)O)[C@H]1CC[C@H]2[C@@H]3C(=O)C[C@@H]4CC(=O)CC[C@]4(C)[C@H]3CC(=O)[C@]12C[END_I_SMILES]

Question: Will the chemical compound penetrate the blood-brain barrier?

Answer: Yes</s>
GT: 0.0 Gen: 1.0 diff: 1.0 Here is a SMILES formula

[START_I_SMILES]Nc1ccc(S(=O)(=O)c2ccc(N)cc2)cc1[END_I_SMILES]

Question: Will the chemical compound penetrate the blood-brain barrier?

Answer: Yes</s>
GT: 1.0 Gen: 1.0 diff: 0.0 Here is a SMILES formula

[START_I_SMILES]NC1[C@H]2CN(c3nc4c(cc3F)c(=O)c(C(=O)O)cn4-c3ccc(F)cc3F)C[C@@H]12[END_I_SMILES]

Question: Will the chemical compound penetrate the blood-brain barrier?

Answer: Yes</s>
GT: 1.0 Gen: 1.0 diff: 0.0 Here is a SMILES formula

[START_I_SMILES]COc1cc(Cc2cnc(N)nc2N)c

In [85]:
sum(diffs)/len(diffs)

0.47549019607843135

In [57]:
from sklearn.metrics import roc_auc_score
roc_auc_score(ground_truths, gens)

0.5

In [1]:
!pip install galai

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 1.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached webencodings-0.5.1-py2.py3-none-any.whl.metadata (2.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 8.9 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.9/160.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.9 MB/s eta 0:00:00
Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
Using cached webencodings-0.5.1-py2.py3-none-any.whl (11 kB)
  Created wheel for galai: filename=galai-1.1.6-py3-none-any.whl size=24725 sha256=a942056d2d7aff9635f9237fb9f6559f8cf5d890dadc9146ed50cdf431583f45
  Stored in directory: /a

In [7]:
import galai as gal
prompt = f"Here is a SMILES formula: [START_I_SMILES]CCOC(=O)[C@H](CCc1ccccc1)N[C@@H](C)C(=O)N2CCC[C@H]2C(O)=O[END_I_SMILES]\n\nQuestion: Will the chemical compound penetrate the blood-brain barrier?\n\nAnswer:"

model = gal.load_model("base")
model.generate(prompt)

'Here is a SMILES formula: [START_I_SMILES]CCOC(=O)[C@H](CCc1ccccc1)N[C@@H](C)C(=O)N2CCC[C@H]2C(O)=O[END_I_SMILES]\n\nQuestion: Will the chemical compound penetrate the blood-brain barrier?\n\nAnswer: Yes'

In [97]:
# tokenizer = galactica_tokenizer
ground_truths, gens, diffs = [],[],[]
invalids = 0
for sample in dataset.values:
    ground_truth = round(sample[1], 2)
    try:
        mol = Chem.MolFromSmiles(sample[2])
        kek_iso_smiles = Chem.MolToSmiles(mol, isomericSmiles=True, kekuleSmiles=True)

        prompt = f"[START_I_SMILES]{kek_iso_smiles}[END_I_SMILES]\n\nQuestion: Will the chemical compound penetrate the blood-brain barrier?\n\nAnswer:"
        # len_prompt = len(prompt)
        # prompt = tokenizer(prompt, return_tensors="pt").to(galactica_model.device)
        # out = galactica_model.generate(prompt.input_ids, do_sample=False, eos_token_id=tokenizer.encode('</s>')[0], max_new_tokens=40)
        # out = tokenizer.batch_decode(out)[0]
        out = model.generate(prompt)
        try:
            gen = out[out.find("Answer: ") + len("Answer: "):out.find("</s>")]
            gen = 0.0 if gen == 'No' else 1.0
            diff = abs(ground_truth - gen)
            print("GT:", ground_truth, "Gen:", gen, "diff:", round(diff,2), out )
            ground_truths.append(ground_truth)
            gens.append(gen)
            diffs.append(diff)
        except:
            print('***')
            print("GT:", ground_truth, out)
            invalids += 1
            pass
    except:
        pass

GT: 1 Gen: 1.0 diff: 0.0 [START_I_SMILES]CC(C)NCC(O)COC1=CC=CC2=CC=CC=C12.[Cl][END_I_SMILES]

Question: Will the chemical compound penetrate the blood-brain barrier?

Answer: Yes
GT: 1 Gen: 1.0 diff: 0.0 [START_I_SMILES]CC(C)(C)OC(=O)CCCC1=CC=C(N(CCCl)CCCl)C=C1[END_I_SMILES]

Question: Will the chemical compound penetrate the blood-brain barrier?

Answer: Yes
GT: 1 Gen: 1.0 diff: 0.0 [START_I_SMILES]CC1COC2=C3C(=CC(F)=C2N2CCN(C)CC2)C(=O)C(C(=O)O)=CN31[END_I_SMILES]

Question: Will the chemical compound penetrate the blood-brain barrier?

Answer: Yes
GT: 1 Gen: 1.0 diff: 0.0 [START_I_SMILES]CC(=O)NCCCOC1=CC(CN2CCCCC2)=CC=C1[END_I_SMILES]

Question: Will the chemical compound penetrate the blood-brain barrier?

Answer: Yes
GT: 1 Gen: 1.0 diff: 0.0 [START_I_SMILES]CC1=C(C(=O)N[C@@H]2C(=O)N3[C@@H]2SC(C)(C)[C@@H]3C(=O)O)C(C2=CC=CC=C2Cl)=NO1[END_I_SMILES]

Question: Will the chemical compound penetrate the blood-brain barrier?

Answer: Yes
GT: 1 Gen: 1.0 diff: 0.0 [START_I_SMILES]CCN1CCN(C(=

[14:44:17] Explicit valence for atom # 1 N, 4, is greater than permitted


ArgumentError: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(RDKit::ROMol mol, RDKit::SmilesWriteParams params)

In [60]:
from sklearn.metrics import roc_auc_score
roc_auc_score(ground_truths, gens)

0.5

In [57]:
0 in gens

False

In [96]:

dataset = pd.read_csv('/auto/home/menuab/code/sft_data/BBBP.csv', index_col='num')

In [50]:
dataset.values


array([['Propanolol', 1, '[Cl].CC(C)NCC(O)COc1cccc2ccccc12'],
       ['Terbutylchlorambucil', 1,
        'C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl'],
       ['40730', 1, 'c12c3c(N4CCN(C)CC4)c(F)cc1c(c(C(O)=O)cn2C(C)CO3)=O'],
       ...,
       ['mesocarb', 1,
        '[O+]1=N[N](C=C1[N-]C(NC2=CC=CC=C2)=O)C(CC3=CC=CC=C3)C'],
       ['tofisoline', 1,
        'C1=C(OC)C(=CC2=C1C(=[N+](C(=C2CC)C)[NH-])C3=CC=C(C(=C3)OC)OC)OC'],
       ['azidamfenicol', 1,
        '[N+](=NCC(=O)N[C@@H]([C@H](O)C1=CC=C([N+]([O-])=O)C=C1)CO)=[N-]']],
      dtype=object)

In [91]:
for mol in test.ids:
    if mol not in dataset2['test']['smiles']:
        print('not htere')

In [43]:
dataset2 = load_dataset('gayane/BBBp')

In [85]:
dataset2['train']['smiles']

['C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl',
 'C[C@H](N)Cc1ccccc1',
 'C[S](=O)(=O)c1ccc(cc1)[C@@H](O)[C@@H](CO)NC(=O)C(Cl)Cl',
 'CC(N)COc1c(C)cccc1C',
 'c1(ccccc1)CC',
 'c1(c(cc(NC(CCC)=O)cc1)C(C)=O)OCC(CNC(C)C)O',
 'CCN(CC)C(=O)Nc1ccc(OCC(O)CNC(C)(C)C)c(c1)C(C)=O',
 'CC(=O)Nc1ccc(O)cc1',
 'CC(=O)Oc1ccccc1C(O)=O',
 'C[C@@](Cc1ccc(O)c(O)c1)(NN)C(O)=O',
 'OC(=O)CCCc1ccc(cc1)N(CCCl)CCCl',
 'NNCCc1ccccc1.O[S](O)(=O)=O',
 'NC(=O)OCCCc1ccccc1',
 'NCC(O)c1ccc(O)c(O)c1',
 'OCC(NC(=O)C(Cl)Cl)C(O)c1ccc(cc1)[N+]([O-])=O',
 'CCCNC(=O)N[S](=O)(=O)c1ccc(Cl)cc1',
 'CCOC(=O)C(C)(C)Oc1ccc(Cl)cc1',
 'CC(C)(Oc1ccc(Cl)cc1)C(O)=O',
 'CCN(CC)CCOC(=O)c1ccc(N)cc1',
 'CC(C)c1cccc(C(C)C)c1O',
 'NCCc1ccc(O)c(O)c1',
 'CN[C@@H](C)[C@H](O)c1ccccc1',
 'CNC[C@H](O)c1ccc(O)c(O)c1',
 'CCC(=C)C(=O)c1ccc(OCC(O)=O)c(Cl)c1Cl',
 'NC(N)=N\\N=C\\c1c(Cl)cccc1Cl',
 'CC(C)Cc1ccc(cc1)C(C)C(O)=O',
 'C[C@H](N)[C@H](O)c1cccc(O)c1',
 'CC(C)NCC(O)c1ccc(O)c(O)c1',
 'N[C@@H](Cc1ccc(O)c(O)c1)C(O)=O',
 'COCCc1ccc(OCC(O)CNC(C)C)cc1',
 'NC(

In [61]:
!pip install deepchem

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.1 MB/s eta 0:00:0000:0100:01


In [64]:
import deepchem as dc
dataset_dc = dc.molnet.load_bbbp()
tasks, dataset, transformers = dataset_dc
train, valid, test = dataset

In [79]:
test.ids

array(['CCOC(=O)[C@H](CCc1ccccc1)N[C@@H](C)C(=O)N2CCC[C@H]2C(O)=O',
       'FC(F)(F)[C@]1(OC(=O)Nc2ccc(Cl)cc12)C#CC3CC3',
       'CN1C[C@@H](C[C@H]2[C@H]1Cc3c[nH]c4cccc2c34)C(=O)N[C@]5(C)O[C@@]6(O)[C@@H]7CCCN7C(=O)[C@H](Cc8ccccc8)N6C5=O',
       'CN1CCC[C@H]1c2cccnc2', 'CN1CCC(=CC1)c2ccccc2',
       'C[C@@H](CN1CC(=O)NC(=O)C1)N2CC(=O)NC(=O)C2',
       'CN(C)N\\N=C/1N=CN=C1C(N)=O', 'CCC1(CC)C(=O)NCC(C)C1=O',
       'Nc1nc(NC2CC2)c3ncn([C@@H]4C[C@H](CO)C=C4)c3n1',
       'c1(c(cc2n(ncc2c1)CCN)Cl)Cl',
       'c1c2CCOc2c(cc1OC)CN[C@@H]1CCCN[C@H]1c1ccccc1',
       'CC(C)N1CCC(CC1)N(C(=O)Cc2ccccc2)c3ccc(Cl)cc3',
       'n1(c(c2nc[nH]c2n(c1=O)C)=O)C', 'c1(nccc(c1)c1nc([nH]n1)N)N(C)C',
       'CN(C)Cc1oc(cc1)CSCCNC=1NC=C(CN1)Cc1cnc(cc1)C',
       'OC(=O)C1=CC(=O)c2ccccc2N1', 'CNC1(CCCCC1=O)c2ccccc2Cl',
       'CN(C1CCCC[C@H]1N2CCCC2)C(=O)Cc3ccc(Cl)c(Cl)c3',
       'CN1C(=C(\\O)Nc2ccccn2)/C(=O)c3sccc3[S]1(=O)=O',
       '[NH]C(CC(C)C([N@@](C(C)(C)C)C(N)(C)N)(C)C)c1c(c(c[nH+][o+]1)C)[O-]',
     